In [68]:
!pip install opencv-python

In [105]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 27 11:30:39 2021

@author: sakalloe
We want to calibrate the camera
step 1: Script detects the corners on chessboard images 

"""
#%% Importing stuff: 
import numpy as np 
import cv2 # Import the OpenCV library to enable computer vision
#%% 
filename = 'DSC00409.JPG'
 
# Chessboard dimensions
number_of_squares_X = 25 # Number of chessboard squares along the x-axis
number_of_squares_Y = 22  # Number of chessboard squares along the y-axis
nX = number_of_squares_X - 1 # Number of interior corners along x-axis
nY = number_of_squares_Y - 1 # Number of interior corners along y-axis
 
def main():
     
  # Load an image
  image = cv2.imread(filename)
 
  # Convert the image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
 
  # Find the corners on the chessboard
  success, corners = cv2.findChessboardCorners(gray, (nY, nX), None)
  print(success)
  # If the corners are found by the algorithm, draw them
  if success == True:
 
    # Draw the corners
    cv2.drawChessboardCorners(image, (nY, nX), corners, success)
 
    # Create the output file name by removing the '.jpg' part
    size = len(filename)
    new_filename = filename[:size - 4]
    new_filename = new_filename + '_drawn_corners.jpg'     
     
    # Save the new image in the working directory
    cv2.imwrite(new_filename, image)
 
    # Display the image 
    cv2.imshow("Image", image) 
     
    # Display the window until any key is pressed
    cv2.waitKey(0) 
     
    # Close all windows
    cv2.destroyAllWindows() 
     
main()

True


In [109]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 27 11:37:50 2021

@author: sakalloe
We want to calibrate the camera
Step 2: Script calibrates the camera using step1 : the corner_detection_chessboard script
"""
#%% Importing stuff: 
import pickle
import numpy as np 
import cv2
import glob # Used to get retrieve files that have a specified pattern
#%% 
distorted_img_filename = 'DSC00409.JPG'
 
# Chessboard dimensions
number_of_squares_X = 22 # Number of chessboard squares along the x-axis
number_of_squares_Y = 25  # Number of chessboard squares along the y-axis
nX = number_of_squares_X - 1 # Number of interior corners along x-axis
nY = number_of_squares_Y - 1 # Number of interior corners along y-axis
 
# Store vectors of 3D points for all chessboard images (world coordinate frame)
object_points = []
 
# Store vectors of 2D points for all chessboard images (camera coordinate frame)
image_points = []
 
# Set termination criteria. We stop either when an accuracy is reached or when
# we have finished a certain number of iterations.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Define real world coordinates for points in the 3D coordinate frame
# Object points are (0,0,0), (1,0,0), (2,0,0) ...., (5,8,0)
object_points_3D = np.zeros((nX * nY, 3), np.float32)       
 
# These are the x and y coordinates                                              
object_points_3D[:,:2] = np.mgrid[0:nY, 0:nX].T.reshape(-1, 2)*1 
 
def main():
     
  # Get the file path for images in the current directory
  images = glob.glob('*.jpg')
     
  # Go through each chessboard image, one by one
  for image_file in images:
  
    # Load the image
    image = cv2.imread(image_file)  
 
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
 
    # Find the corners on the chessboard
    success, corners = cv2.findChessboardCorners(gray, (nY, nX), None)
     
    # If the corners are found by the algorithm, draw them
    if success == True:
 
      # Append object points
      object_points.append(object_points_3D)
 
      # Find more exact corner pixels       
      corners_2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)       
       
            # Append image points
      image_points.append(corners)
 
      # Draw the corners
      cv2.drawChessboardCorners(image, (nY, nX), corners_2, success)
 
      # Display the image. Used for testing.
      #cv2.imshow("Image", image) 
     
      # Display the window for a short period. Used for testing.
      #cv2.waitKey(200) 
                                                                                                                     
  # Now take a distorted image and undistort it 
  distorted_image = cv2.imread(distorted_img_filename)
 
  # Perform camera calibration to return the camera matrix, distortion coefficients, rotation and translation vectors etc 
  ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, 
                                                    image_points, 
                                                    gray.shape[::-1], 
                                                    None, 
                                                    None)
 
  # Get the dimensions of the image 
  height, width = distorted_image.shape[:2]
     
  # Refine camera matrix
  # Returns optimal camera matrix and a rectangular region of interest
  optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, 
                                                            (width,height), 
                                                            1, 
                                                            (width,height))
 
  # Undistort the image
  undistorted_image = cv2.undistort(distorted_image, mtx, dist, None, 
                                    optimal_camera_matrix)
   
  # Crop the image. Uncomment these two lines to remove black lines
  # on the edge of the undistorted image.
  #x, y, w, h = roi
  #undistorted_image = undistorted_image[y:y+h, x:x+w]
     
  # Display key parameter outputs of the camera calibration process
  print("Optimal Camera matrix:") 
  print(optimal_camera_matrix) 
 
  print("\n Distortion coefficient:") 
  print(dist) 
   
  print("\n Rotation Vectors:") 
  print(rvecs) 
   
  print("\n Translation Vectors:") 
  print(tvecs) 
 
  # Create the output file name by removing the '.jpg' part
  size = len(distorted_img_filename)
  new_filename = distorted_img_filename[:size - 4]
  new_filename = new_filename + '_undistorted.jpg'
     
  # Save the undistorted image
  cv2.imwrite(new_filename, undistorted_image)
 
  # Close all windows
  cv2.destroyAllWindows() 
  
  # Saving the important parameters using Pickle. 
  calib_result_pickle = {}
  calib_result_pickle["mtx"] = mtx
  calib_result_pickle["optimal_camera_matrix"] = optimal_camera_matrix
  calib_result_pickle["dist"] = dist
  calib_result_pickle["rvecs"] = rvecs
  calib_result_pickle["tvecs"] = tvecs
  pickle.dump(calib_result_pickle, open("camera_calib_pickle.p", "wb" )) 

     
main()


#%% If i want to use pickle to open the results:
# calib_result_pickle = pickle.load(open("camera_calib_pickle.p", "rb" ))
# mtx = calib_result_pickle["mtx"]
# optimal_camera_matrix = calib_result_pickle["optimal_camera_matrix"]
# dist = calib_result_pickle["dist"]

# #So given an new video or image , we can undistort it using:
# undistorted_image = cv2.undistort(distorted_image, 
#                                   mtx, dist, 
#                                   None, 
#                                   optimal_camera_matrix)

Optimal Camera matrix:
[[2.21893994e+03 0.00000000e+00 1.05927803e+03]
 [0.00000000e+00 2.19462988e+03 5.52918088e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 Distortion coefficient:
[[ 0.07696089  0.0752307   0.00186435  0.0127851  -1.00909004]]

 Rotation Vectors:
[array([[-0.03666718],
       [-0.09107386],
       [ 0.00675407]]), array([[ 0.01038691],
       [-0.11800363],
       [-0.26545022]]), array([[-0.07598289],
       [-0.11221985],
       [-0.00613408]]), array([[-0.27928516],
       [-0.03856   ],
       [ 0.00105376]]), array([[-0.15698253],
       [-0.05861988],
       [ 0.07158267]]), array([[-0.08559074],
       [-0.04401512],
       [-0.01631121]]), array([[ 0.00723329],
       [-0.08572483],
       [-0.02289219]]), array([[-0.01908807],
       [-0.10378036],
       [ 0.20702163]]), array([[ 0.04246906],
       [-0.13323094],
       [-0.35871456]]), array([[ 0.03098913],
       [ 0.15228231],
       [-0.26905115]]), array([[ 0.05091717],
       [ 0.23008306